## Load necessary modules

In [1]:
import numpy as np
# Malisiewicz et al.
def non_max_suppression_fast(boxes, overlapThresh):
    # if there are no boxes, return an empty list
    if len(boxes) == 0:
        return []

    # if the bounding boxes integers, convert them to floats --
    # this is important since we'll be doing a bunch of divisions
    if boxes.dtype.kind == "i":
        boxes = boxes.astype("float")

    # initialize the list of picked indexes	
    pick = []

    # grab the coordinates of the bounding boxes
    x1 = boxes[:,0]
    y1 = boxes[:,1]
    x2 = boxes[:,2]
    y2 = boxes[:,3]

    # compute the area of the bounding boxes and sort the bounding
    # boxes by the bottom-right y-coordinate of the bounding box
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(y2)

    # keep looping while some indexes still remain in the indexes
    # list
    while len(idxs) > 0:
        # grab the last index in the indexes list and add the
        # index value to the list of picked indexes
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)

        # find the largest (x, y) coordinates for the start of
        # the bounding box and the smallest (x, y) coordinates
        # for the end of the bounding box
        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])

        # compute the width and height of the bounding box
        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)

        # compute the ratio of overlap
        overlap = (w * h) / area[idxs[:last]]

        # delete all indexes from the index list that have
        idxs = np.delete(idxs, np.concatenate(([last],
            np.where(overlap > overlapThresh)[0])))

    # return only the bounding boxes that were picked using the
    # integer data type
    return boxes[pick].astype("int")

In [2]:
# show images inline
%matplotlib inline

# automatically reload modules when they have changed
%load_ext autoreload
%autoreload 2

# import keras
import keras

# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color

# import miscellaneous modules
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import time

# set tf backend to allow memory to grow, instead of claiming everything
import tensorflow as tf

def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)

# use this environment flag to change which GPU to use
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# set the modified tf session as backend in keras
keras.backend.tensorflow_backend.set_session(get_session())

Using TensorFlow backend.


## Sequence init

In [1]:
from keras_retinanet.preprocessing.alan import TestSequence

Using TensorFlow backend.


In [5]:
a = TestSequence()

Img and gt loading: 100%|█████████████████████████| 1/1 [00:00<00:00, 12.16it/s]

['/dds/work/workspace/alan_jpg_files/SAL1']
Saving this scan
                Nb img : 1
              Nb crops : 578 [578 crops/img]
   Nb crops with obj : 24
Nb crops without obj : 554
Using all crops for evaluation [ 578 ].


In [6]:
a.__getitem__(0)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Error in sample_index /dds/work/workspace/alan_jpg_files/SAL1/Niveau 6/PANO_SAL1_BR_6201.jpg 0 1 (0, 0, 1000, 1000) False
Traceback (most recent call last):
  File "/home/j61678/soft/anaconda3/envs/retinanet1/lib/python3.6/site-packages/keras_retinanet/preprocessing/alan.py", line 91, in load_crop
    return img.crop((xywh[1],xywh[0],xywh[1]+xywh[2],xywh[0]+xywh[3]))
  File "/home/j61678/soft/anaconda3/envs/retinanet1/lib/python3.6/site-packages/PIL/Image.py", line 1090, in crop
    self.load()
  File "/home/j61678/soft/anaconda3/envs/retinanet1/lib/python3.6/site-packages/PIL/ImageFile.py", line 235, in load
    n, err_code = decoder.decode(b)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/j61678/soft/anaconda3/envs/retinanet1/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-82c105a

TypeError: must be str, not list

In [ ]:
# generator = ALANGenerator()
# import random
# len(generator.img_infos['crops_without_objs'])
# train_empty = random.sample(generator.img_infos['crops_without_objs'],1000)
# generator.data = train_empty

## Load RetinaNet model

In [ ]:
# adjust this to point to your downloaded/trained model
# models can be downloaded here: https://github.com/fizyr/keras-retinanet/releases
model_path = os.path.join('..', 'snapshots', 'resnet50_coco_best_v2.1.0.h5')
model_path = '/dds/work/workspace/alan_tmp_files/resnet50_alan_05_inference.h5'

# load retinanet model
model = models.load_model(model_path, backbone_name='resnet50')

# if the model is not converted to an inference model, use the line below
# see: https://github.com/fizyr/keras-retinanet#converting-a-training-model-to-inference-model
#model = models.load_model(model_path, backbone_name='resnet50', convert=True)

#print(model.summary())

# load label to names mapping for visualization purposes
labels_to_names = {0: 'etiquette'}

## Run detection on example

In [ ]:
i=-1

In [ ]:
# load image
# img_path = '/dds/work/workspace/alan_tmp_files/Image_screenshot_31.07.2018_2.png'
# img_path = '/dds/work/workspace/alan_tmp_files/Image_screenshot_31.07.2018_3.png'
# img_path = '/dds/work/workspace/alan_tmp_files/Image_screenshot_31.07.2018.png'
# image = read_image_bgr(img_path)
i+=1
image = generator.load_image(i)


# image = read_image_bgr('000000008021.jpg')

# copy to draw on
draw = image.copy()
draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

# preprocess image for network
image = preprocess_image(image)
image, scale = resize_image(image)

# process image
start = time.time()
boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
print("processing time: ", time.time() - start)

# correct for image scale
boxes /= scale

boxes = np.expand_dims(non_max_suppression_fast(boxes[0],0.5), axis=0)

# visualize detections
for box, score, label in zip(boxes[0], scores[0], labels[0]):
    # scores are sorted so we can break
    if score < 0.01:
        break
        
    color = label_color(label)
    
    b = box.astype(int)
    draw_box(draw, b, color=color)
    
    caption = "{} {:.3f}".format(labels_to_names[label], score)
    draw_caption(draw, b, caption)
    
plt.figure(figsize=(15, 15))
plt.axis('off')
plt.imshow(draw)
plt.show()